In [ ]:
import json
import nltk
import numpy as np
import scipy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import OrderedDict

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


lemmatizer = WordNetLemmatizer()
tfidf_vectorizer = TfidfVectorizer()
testing = {}
documents = {}

# to store all documents

with open('documents.json') as f:
    docs = json.load(f) 
for doc in docs:
    documents[doc['docid']] = doc['text']

with open('testing.json') as f:
    test_set = json.load(f)
for test in test_set:
#     if len(testing) < 200:
    testing[test['question']] = [test['docid'],test['id']]
    
def clean_filter(sents):
   tokens = nltk.word_tokenize(sents)
   tokens_lowered = [item.lower() for item in tokens]
   # Lemmatizing the words and selecting non stop words
   tokens_filtered = [lemmatizer.lemmatize(token) for token in tokens_lowered if token not in stopwords.words('english')]
   # Selecting only the word types

   return tokens_filtered

final_ques = []
final_para = []
final_ques_id = []
accuracy = 0.0
match = 0.0
count = len(testing.keys())
print("Number of testing questions", count)
doc_test = {}
training_test = {}
# Stores all the documents
doc_test = documents 

i = 0.0

for key in testing.keys():
    print(key)
    print(i)
    i = i+1

    query = key
    final_ques.append(query)
    final_ques_id.append(testing[key][1])

    train_set = [" ".join(clean_filter(query))]
    
    for key1 in doc_test.keys():        
        # Check the number of document for the question
        if testing[key][0] == key1:
            for para_index, para in enumerate(doc_test[key1]):
                clean_set = " ".join(clean_filter(para))
                train_set.append(' '.join(clean_filter(para)))

            tfidf_matrix_train = tfidf_vectorizer.fit_transform(train_set)  # finds the tfidf score with normalization
            a = cosine_similarity(tfidf_matrix_train[0:1], tfidf_matrix_train)

            # here the first element of tfidf_matrix_train is matched with other three elements
            updated_a = np.delete(a, 0)
            indices = np.argmax(updated_a)
            testing[key].append(indices)
            training_test[key] = [testing[key][1],indices]
            final_para.append(doc_test[key1][indices])
            break

final = [final_para, final_ques, final_ques_id]
# accuracy = match / count * 100
# print(accuracy)

print(final)


In [ ]:
questionwords = ["who", "what", "where", "when", "why", "how", "whose", "which", "whom"]
commonwords = ["the", "a", "an", "is", "are", "were", "."]
quantifiers = ["few", "little", "much", "many"]
time_quant = ["young", "old", "long", "earliest", "ago", "next", "last", "before", "after", "year"]
percent = ["percent", "percentage"]
company = ["university", "company", "school", "organization", "organisation", "acquisition"]

def processquestion(qwords):
    # Find "question word" (what, who, where, etc.)
    questionword = ""

    qwords = [item.encode('ascii', 'ignore') for item in qwords]

    for (index, word) in enumerate(qwords):
        if word.lower() in questionwords:
            questionword = word.lower()
            
    print("question words are", questionword)
    
        
    type_of_ques = " "
    
    # Determine question type
    if questionword in ["who", "whose", "whom"]:
        type_of_ques = "PERSON"
    elif questionword in ["where", "which", "country", "community", "region"]:
        type_of_ques = "PLACE"
    elif questionword in ["when"]:
        type_of_ques = "TIME"
    elif questionword == "how":
        if not set(quantifiers).isdisjoint(qwords):
            type_of_ques = "QUANTITY"
        elif not set(time_quant).isdisjoint(qwords):
            type_of_ques = "TIME"
    elif not set(time_quant).isdisjoint(qwords):
         type_of_ques = "TIME"
    elif not set(percent).isdisjoint(qwords):
        type_of_ques = "PERCENT"
    elif not set(company).isdisjoint(qwords):
        type_of_ques = "ORGANIZATION"
    else:
        type_of_ques = "MISC"

    # Trim possible extra helper verb
    if questionword == "which":
        target = qwords
    target = qwords
    
    # Return question data
    return (type_of_ques, target)


In [ ]:
import json
import nltk
import numpy as np
import scipy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import OrderedDict

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import collections 
from nltk.parse.stanford import StanfordParser
from nltk import sent_tokenize

from corenlp import *

corenlp = StanfordCoreNLP()

final_results = {}

for (index,question) in enumerate(final[1]):
    print("query is:", question)
    done = False
            
    qwords = nltk.word_tokenize(question.replace('?', ''))
    questionPOS = nltk.pos_tag(qwords)
        
    (type_of_ques, target) = processquestion(qwords)
    
    print(type_of_ques)
    
    ques_id = final[2][index]
    
    # Get sentence keywords
    searchwords = set(target).difference(commonwords)
    diction = collections.Counter()
        
    split_para = nltk.sent_tokenize(final[0][index])
                
    for (i, sent) in enumerate(split_para):
        print("inside here---1")
        sentwords = nltk.word_tokenize(sent)
        wordmatches = set(filter(set(searchwords).__contains__, sentwords)) # words common in query and para
        diction[sent] = len(wordmatches)
                    
    # Focus on 3 most relevant sentences
    for (sentence, matches) in diction.most_common(3):
                
        print("inside here---2")
        
        try:
            sentence = sentence.encode("ascii", errors = "ignore")
            parse = json.loads(corenlp.parse(sentence))
            sentencePOS = nltk.pos_tag(nltk.word_tokenize(sentence))

            # Check if solution is found
            if done:
                continue

            # Check by question type
            answer = ""
            for worddata in parse["sentences"][0]["words"]:  
                if type_of_ques == "PERSON":
                    if worddata[1]["NamedEntityTag"] == "PERSON":
                        if answer == "":
                            answer = worddata[0]
                        else:
                            answer = answer + " " + worddata[0]
                        done = True
                    elif done:
                        break

                if type_of_ques == "PLACE":
                    if worddata[1]["NamedEntityTag"] == "LOCATION":
                        if answer == "":
                            answer = worddata[0]
                        else:
                            answer = answer + " " + worddata[0]
                        done = True
                    elif done:
                        break

                elif type_of_ques == "QUANTITY":
                    if worddata[1]["NamedEntityTag"] == "NUMBER":
                        if answer == "":
                            answer = worddata[0]
                        else:
                            answer = answer + " " + worddata[0]
                        done = True
                    elif done:
                        break

                elif type_of_ques == "TIME":
                    if worddata[1]["NamedEntityTag"] == "DATE":
                        if answer == "":
                            answer = worddata[0]
                        else:
                            answer = answer + " " + worddata[0]
                        done = True
                    elif done:
                        answer = answer + " " + worddata[0]
                        break

                elif type_of_ques == "PERCENT":
                    if worddata[1]["NamedEntityTag"] == "PERCENT":
                        if answer == "":
                            answer == worddata[0]
                        else:
                            answer = answer + " " + worddata[0]
                        done = True
                    elif done:
                        answer = answer + " " + worddata[0]
                        break
                        
                elif type_of_ques == "ORGANIZATION":
                    if worddata[1]["NamedEntityTag"] == "ORGANIZATION":
                        if answer == "":
                            answer == worddata[0]
                        else:
                            answer = answer + " " + worddata[0]
                        done = True
                    elif done:
                        answer = answer + " " + worddata[0]
                        break
             
            if not done:
                (answer, matches) = diction.most_common(1)[0]
        
        except Exception:
                pass

    answer = answer.encode('ascii', 'ignore')
    answer = answer.lower()
    answer = answer.strip()
    
    ques_id = (str(ques_id)).encode('ascii', 'ignore')

    print("The answer is", answer.encode('ascii', 'ignore'))
    final_results[question.encode('ascii', 'ignore')] = [answer, ques_id]
print(final_results)


In [ ]:
new_dict = {}

for item in range(len(final_results)):
    for k,v in final_results.iteritems():
        new_dict.update({int(v[1]): v[0]})

import collections

od = collections.OrderedDict(sorted(new_dict.items()))

import csv
with open('results.csv', "wb") as csv_file:
    writer = csv.writer(csv_file, skipinitialspace=True, delimiter=',', quoting=csv.QUOTE_NONE, escapechar=' ')
    
    writer.writerow(["id", "answer"])
    
    for key,value in od.iteritems():
        writer.writerow([key, value.replace(",", "")])